In [91]:
import pandas as pd
from pathlib import Path
import numpy as np
import alpaca_trade_api as tradeapi
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame

from finta import TA
from finta.utils import resample_calendar
import json
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

# Alpaca for data
import alpaca_trade_api as api
from alpaca_trade_api.rest import TimeFrame

# pandas for analysis
import pandas as pd

# Plotly for charting
import plotly.graph_objects as go
import plotly.express as px

# Set default charting for pandas to plotly
pd.options.plotting.backend = "plotly"

import datetime as dt
import pytz

import os
import requests



In [92]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [93]:
load_dotenv()

True

In [94]:
# Our API keys for Alpaca
API_KEY = os.getenv('ALPACA_API_KEY')

API_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')

In [95]:
def alpaca_data_import(coin):

    # Load .env environment variables

    # Our API keys for Alpaca
    API_KEY = os.getenv('ALPACA_API_KEY')

    API_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')

    #today = dt.date.today()
    # symbols we will be looking at
    #btc = "BTCUSD"
    #spy = "SPY"

    # start dates and end dates for backtest
    #start_date = "2020-01-01"
    #end_date = today 
    #end_date = '2023-02-06'

    end_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(hours=1)).isoformat()

    start_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(days=1080)).isoformat()

    # time frame for backtests
    timeframe = TimeFrame.Day

    # Setup instance of alpaca api
    alpaca = api.REST(API_KEY, API_SECRET_KEY)

    # # # Request historical bar data for SPY and BTC using Alpaca Data API
    # for equities, use .get_bars
    #spy_data = alpaca.get_bars(spy, timeframe, start_date, end_date).df

    # for crypto, use .get_crypto_bars, from multiple exchanges
    #btc_data = alpaca.get_crypto_bars(btc, timeframe, start_date, end_date).df
    df = alpaca.get_crypto_bars(coin, timeframe, start_date, end_date).df

    df = df[df['exchange'] == 'CBSE']

    # display crypto bar data
    display(df)
    #display(spy_data)

    return df

In [96]:
btc_data = alpaca_data_import('BTCUSD')

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2020-03-01 06:00:00+00:00,CBSE,8522.30,8656.23,8487.33,8655.51,2329.079350,15138,8596.448236
2020-03-02 06:00:00+00:00,CBSE,8655.50,8973.45,8591.22,8915.00,7887.291834,47416,8824.567644
2020-03-03 06:00:00+00:00,CBSE,8856.17,8924.69,8635.31,8757.84,7221.564782,42315,8756.636686
2020-03-04 06:00:00+00:00,CBSE,8759.99,8958.11,8759.99,8937.43,2688.206136,15549,8886.647385
2020-03-05 06:00:00+00:00,CBSE,8937.42,9174.24,8894.64,9070.17,7168.234797,41233,9078.918061
...,...,...,...,...,...,...,...,...
2023-02-09 06:00:00+00:00,CBSE,22605.13,22832.44,21636.10,21802.71,16887.384495,382351,22184.534472
2023-02-10 06:00:00+00:00,CBSE,21802.72,21934.31,21467.43,21678.33,10756.288232,289478,21720.292366
2023-02-11 06:00:00+00:00,CBSE,21678.34,21904.00,21620.91,21803.77,4520.042871,188230,21738.122978


In [97]:

# #today = dt.date.today()
# # symbols we will be looking at
# btc = "BTCUSD"
# #spy = "SPY"

# # start dates and end dates for backtest
# #start_date = "2020-01-01"
# #end_date = today 
# #end_date = '2023-02-06'

# end_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(hours=1)).isoformat()

# start_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(days=1080)).isoformat()

# # time frame for backtests
# timeframe = TimeFrame.Day

In [98]:
# # Our API keys for Alpaca
# API_KEY = os.getenv('ALPACA_API_KEY')

# API_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')

# # Setup instance of alpaca api
# alpaca = api.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)

# # # # Request historical bar data for SPY and BTC using Alpaca Data API
# # for equities, use .get_bars
# #spy_data = alpaca.get_bars(spy, timeframe, start_date, end_date).df

# # for crypto, use .get_crypto_bars, from multiple exchanges
# btc_data = alpaca.get_crypto_bars(btc, timeframe, start_date, end_date).df

# btc_data = btc_data[btc_data['exchange'] == 'CBSE']

# # display crypto bar data
# display(btc_data)
# #display(spy_data)

In [99]:
def process_data_ohlcv(df):

    ohlcv_df = df.drop(columns=['exchange','trade_count','vwap'])
    #ohlc_df = df.drop(columns=['volume'])
    ichimoku_df = TA.ICHIMOKU(ohlcv_df, tenkan_period= 20, kijun_period= 60, senkou_period= 120, chikou_period= 30)
    #ETH ichimoku_df = TA.ICHIMOKU(ohlcv_df, tenkan_period= 13, kijun_period= 34, senkou_period= 55, chikou_period= 21)

    ichimoku_df = pd.concat([ichimoku_df, ohlcv_df['close']], axis=1)

    return ichimoku_df
#ohlcv_df

#ohlcv_df = resample_calendar(ohlcv_df, '4h')
#ohlcv_df = ohlcv_df.dropna()


In [100]:
ichimoku_df = process_data_ohlcv(btc_data)
#ichimoku_df

In [101]:
def get_signal(df):
    
    df['signal'] = 0
    df["signal"] = np.where((df["TENKAN"] > df["KIJUN"]) &
                                (df["close"] > df["senkou_span_a"]), 1, 0)

    df['entry/exit'] = df['signal'].diff()
    
    df['actual_returns'] = df['close'].pct_change()
    df["strategy_returns"] = df["actual_returns"] * df["signal"].shift() 

    actual_returns = df['actual_returns'].cumsum()
    
    
    return df

In [102]:
ichimoku_signal_df =  get_signal(ichimoku_df)
#ichimoku_signal_df
sell_signals_df = ichimoku_signal_df['entry/exit'] == -1.0
sell_signals_df.tail(100)

timestamp
2022-11-06 05:00:00+00:00    False
2022-11-07 06:00:00+00:00    False
2022-11-08 06:00:00+00:00    False
2022-11-09 06:00:00+00:00    False
2022-11-10 06:00:00+00:00    False
                             ...  
2023-02-09 06:00:00+00:00    False
2023-02-10 06:00:00+00:00    False
2023-02-11 06:00:00+00:00    False
2023-02-12 06:00:00+00:00    False
2023-02-13 06:00:00+00:00    False
Name: entry/exit, Length: 100, dtype: bool

In [103]:
def create_order(symbol, qty, side, order_type, time_in_force, api_key_id, api_secret_key):
    headers = {
        "Apca-Api-Key-Id": api_key_id,
        "Apca-Api-Secret-Key": api_secret_key,
        "Content-Type": "application/json"
    }

    data = {
        "symbol": symbol,
        "qty": qty,
        "side": side,
        "type": order_type,
        "time_in_force": time_in_force
    }

    url = "https://paper-api.alpaca.markets/v2/orders"
    response = requests.post(url, headers=headers, json=data)

    if response.status_code != 200:
        raise ValueError("Failed to create order: " + response.text)

    return response.json()


In [104]:

api_key_id = API_KEY
api_secret_key = API_SECRET_KEY

#result = create_order("BTC/USD", "1.0", "buy", "market", "gtc", api_key_id, api_secret_key)
#print(result)


In [105]:
ichimoku_signal_df['entry/exit'].iloc[-1] 

0.0

In [106]:
if ichimoku_signal_df['entry/exit'].iloc[-1] == 1.0:
    create_order("BTC/USD", "1.0", "buy", "market", "gtc", api_key_id, api_secret_key)
elif ichimoku_signal_df['entry/exit'].iloc[-1] == -1.0:
    create_order("BTC/USD", "1.0", "sell", "market", "gtc", api_key_id, api_secret_key)
else:
    print('No Trades Today!')

No Trades Today!


In [107]:
# ohlc_df = ohlcv_df.drop(columns=['volume'])
# #ohlc_df

In [108]:
# ichimoku_df = TA.ICHIMOKU(ohlcv_df, tenkan_period= 20, kijun_period= 60, senkou_period= 120, chikou_period= 30)
# #ichimoku_df

In [135]:

ichimoku_signal_df.tail(35)

,TENKAN,KIJUN,senkou_span_a,SENKOU,CHIKOU,close,signal,entry/exit,actual_returns,strategy_returns
timestamp,,,,,,,,,,
2023-01-10 06:00:00+00:00,16918.225,16922.680,18825.7000,20363.285,21802.71,17409.60,0,0.0,0.010663,0.000000
2023-01-11 06:00:00+00:00,17350.920,16922.680,18495.4000,20363.285,21678.33,18085.57,0,0.0,0.038827,0.000000
2023-01-12 06:00:00+00:00,17721.005,17287.925,18495.4000,20363.285,21803.77,18790.58,1,1.0,0.038982,0.000000
2023-01-13 06:00:00+00:00,18824.070,18390.990,18495.4000,20363.285,21839.49,20843.55,1,0.0,0.109255,0.109255
2023-01-14 06:00:00+00:00,18824.070,18390.990,18495.4000,20363.285,21604.61,20752.70,1,0.0,-0.004359,-0.004359
2023-01-15 06:00:00+00:00,18881.590,18448.510,18495.4000,20363.285,NaN,21198.17,1,0.0,0.021466,0.021466
2023-01-16 06:00:00+00:00,18895.890,18462.810,18495.4000,20363.285,NaN,21151.25,1,0.0,-0.002213,-0.002213
2023-01-17 06:00:00+00:00,18960.580,18527.500,18495.4000,20363.285,NaN,21312.81,1,0.0,0.007638,0.007638
2023-01-18 06:00:00+00:00,18988.185,18555.105,18495.4000,20363.285,NaN,20812.48,1,0.0,-0.023476,-0.023476


In [110]:

# ichimoku_signal_df['signal'] = 0
# ichimoku_signal_df["signal"] = np.where((ichimoku_signal_df["TENKAN"] > ichimoku_signal_df["KIJUN"]) &
#                                 (ichimoku_signal_df["close"] > ichimoku_signal_df["senkou_span_a"]), 1, 0)

# ichimoku_signal_df['entry/exit'] = ichimoku_signal_df['signal'].diff()

# #ichimoku_signal_df


In [111]:
# ichimoku_signal_df['actual_returns'] = ichimoku_signal_df['close'].pct_change()
# ichimoku_signal_df["strategy_returns"] = ichimoku_signal_df["actual_returns"] * ichimoku_signal_df["signal"].shift() 


In [112]:
#ichimoku_signal_df

In [113]:


def get_open_positions(api_key_id, api_secret_key):
    # Initialize the Alpaca API client
   # api = tradeapi.REST(api_key_id, api_secret_key, api_version='v2')
    api = tradeapi.REST(api_key_id, api_secret_key, base_url='https://paper-api.alpaca.markets', api_version='v2')

    # Get a list of all open positions
    open_positions = api.list_positions()
    
    # Return the list of open positions
    
    return open_positions


In [114]:
open_positions = get_open_positions(api_key_id, api_secret_key)


In [115]:
open_positions

[Position({   'asset_class': 'us_equity',
     'asset_id': '8ccae427-5dd0-45b3-b5fe-7ba5e422c766',
     'asset_marginable': False,
     'avg_entry_price': '143.04',
     'change_today': '-0.0099547970948245',
     'cost_basis': '14304',
     'current_price': '194.93',
     'exchange': 'NASDAQ',
     'lastday_price': '196.89',
     'market_value': '19493',
     'qty': '100',
     'qty_available': '0',
     'side': 'long',
     'symbol': 'TSLA',
     'unrealized_intraday_pl': '-196',
     'unrealized_intraday_plpc': '-0.0099547970948245',
     'unrealized_pl': '5189',
     'unrealized_plpc': '0.3627656599552573'})]

In [116]:
open_positions = get_open_positions(api_key_id, api_secret_key)
open_positions



[Position({   'asset_class': 'us_equity',
     'asset_id': '8ccae427-5dd0-45b3-b5fe-7ba5e422c766',
     'asset_marginable': False,
     'avg_entry_price': '143.04',
     'change_today': '-0.0099547970948245',
     'cost_basis': '14304',
     'current_price': '194.93',
     'exchange': 'NASDAQ',
     'lastday_price': '196.89',
     'market_value': '19493',
     'qty': '100',
     'qty_available': '0',
     'side': 'long',
     'symbol': 'TSLA',
     'unrealized_intraday_pl': '-196',
     'unrealized_intraday_plpc': '-0.0099547970948245',
     'unrealized_pl': '5189',
     'unrealized_plpc': '0.3627656599552573'})]

In [117]:
#actual_returns = ichimoku_signal_df['actual_returns'].cumsum()

actual_returns_plot =  (1 + ichimoku_signal_df[['actual_returns']]).cumprod().hvplot(
    color='lightblue'
)

In [118]:
strategy_returns_plot = (1 + ichimoku_signal_df[['strategy_returns']]).cumprod().hvplot(
    color='lightgreen'
)

In [119]:
#ichimoku_signal_df

In [120]:
#ichimoku_signal_df['entry/exit'].value_counts()

In [121]:
entry = ichimoku_signal_df[ichimoku_signal_df['entry/exit']==1]['close'].hvplot.scatter(
    color='green',
    marker= '^'
)

exit = ichimoku_signal_df[ichimoku_signal_df['entry/exit']==-1]['close'].hvplot.scatter(
    color='red',
    marker= 'v'
)

close = ichimoku_signal_df['close'].hvplot(
    color='lightgray',
    
)

#actual_returns = actual_returns.hvplot(
 #   color='lightblue'
#)

#strategy_returns = ichimoku_signal_df['strategy_returns'].hvplot(
 #   color='lightgreen',
#)

In [122]:
ichimoku_signal_df['entry/exit'].value_counts()

 0.0    1064
 1.0       8
-1.0       7
Name: entry/exit, dtype: int64

In [123]:
ichimoku_signal_df.columns

Index(['TENKAN', 'KIJUN', 'senkou_span_a', 'SENKOU', 'CHIKOU', 'close',
       'signal', 'entry/exit', 'actual_returns', 'strategy_returns'],
      dtype='object')

In [124]:
close * entry * exit

:Overlay
   .Curve.Close      :Curve   [timestamp]   (close)
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [125]:
actual_returns_plot * strategy_returns_plot

:Overlay
   .Curve.I  :Curve   [timestamp]   (actual_returns)
   .Curve.II :Curve   [timestamp]   (strategy_returns)

In [126]:
strategy_returns_annual_volitility = ichimoku_signal_df[['strategy_returns']].std()*np.sqrt(365)
strategy_returns_annual_volitility



strategy_returns    0.456065
dtype: float64

In [127]:
annualized_return = ichimoku_signal_df["strategy_returns"].mean() * 365
annualized_std = ichimoku_signal_df["strategy_returns"].std() * np.sqrt(365)
sharpe_ratio = round(annualized_return/annualized_std, 3)


print(f"ichimoku strategy annualized returns: {round(annualized_return,2)}")
print(f"ichimoku strategy annualized volitility: {round(annualized_std,2)}")
print(f"ichimoku strategy sharpe ratio: {sharpe_ratio}")

ichimoku strategy annualized returns: 0.61
ichimoku strategy annualized volitility: 0.46
ichimoku strategy sharpe ratio: 1.339


In [128]:
# ichimoku_df.hvplot()

In [129]:
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

def get_trends(keywords):
    pytrends.build_payload(kw_list=keywords, timeframe='today 12-m')  
    interest_over_time_data = pytrends.interest_over_time()
    return interest_over_time_data

keywords = ['bitcoin']
trends_data = get_trends(keywords)
print(trends_data)


            bitcoin  isPartial
date                          
2022-02-13       46      False
2022-02-20       60      False
2022-02-27       55      False
2022-03-06       48      False
2022-03-13       43      False
2022-03-20       41      False
2022-03-27       43      False
2022-04-03       39      False
2022-04-10       37      False
2022-04-17       34      False
2022-04-24       35      False
2022-05-01       35      False
2022-05-08       86      False
2022-05-15       55      False
2022-05-22       45      False
2022-05-29       39      False
2022-06-05       38      False
2022-06-12      100      False
2022-06-19       76      False
2022-06-26       55      False
2022-07-03       51      False
2022-07-10       45      False
2022-07-17       47      False
2022-07-24       41      False
2022-07-31       36      False
2022-08-07       34      False
2022-08-14       33      False
2022-08-21       33      False
2022-08-28       33      False
2022-09-04       33      False
2022-09-

In [130]:
trends_data_plot =  trends_data['bitcoin'].hvplot(
    color='red',
    title='Bitcoin Google Trends, Weekly',
    
)

In [131]:
trends_data_plot

:Curve   [date]   (bitcoin)